In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import pickle
import numpy as np
from tensorflow.keras.callbacks import TensorBoard
import time


2025-12-18 23:39:52.116459: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-18 23:39:52.760358: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-18 23:39:55.030788: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
print(tf.__version__)

2.20.0


In [3]:
# Place this at the very top of your script, right after imports
import tensorflow as tf

# 1. Check if a GPU is detected
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # 2. Configure memory growth (better than a fixed fraction)
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # 3. Restrict TensorFlow to only use the first GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        print(f"GPUs detected and configured: {gpus[0].name}")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU detected. Falling back to CPU.")


GPUs detected and configured: /physical_device:GPU:0


In [4]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        print(f"GPUs detected and configured: {gpus[0].name}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU detected. Falling back to CPU.")

X = pickle.load(open("/home/nigga/engine/X.pickle", "rb"))
y = pickle.load(open("/home/nigga/engine/y.pickle", "rb"))

X = X / 255.0
dense_layers = [0,1,2]
layer_sizes = [64,128,256]
conv_layers  = [1,2,3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME="{}-conv-{}-nodes-{}-dense-{}".format(conv_layer,layer_size,dense_layer,int(time.time()))
            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
            print(NAME)
        
            model = Sequential()            
            model.add(Conv2D(layer_size, (3,3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2,2)))
            for l in range(conv_layer-1): 
                model.add(Conv2D(layer_size, (3,3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2,2)))
            model.add(Flatten())
            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))
            model.add(Dense(1, activation='sigmoid'))            
            # Compile
            model.compile(
                loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy']
            )
            
            X = np.array(X)
            y = np.array(y)
            model.fit(X, y, batch_size=32, epochs=20, validation_split=0.3,callbacks=[tensorboard])


GPUs detected and configured: /physical_device:GPU:0
1-conv-64-nodes-0-dense-1766097619


/home/nigga/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1766097620.091780     452 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3584 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/20


2025-12-18 23:40:23.682439: I external/local_xla/xla/service/service.cc:163] XLA service 0x721030008940 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-18 23:40:23.682523: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2025-12-18 23:40:23.713961: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-18 23:40:23.868328: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91600


 17/479 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.4519 - loss: 1.0835

...

479/479 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - accuracy: 0.6667 - loss: 0.6208 - val_accuracy: 0.7331 - val_loss: 0.5368
Epoch 2/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.7532 - loss: 0.5067 - val_accuracy: 0.7886 - val_loss: 0.4537
Epoch 3/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.7996 - loss: 0.4376 - val_accuracy: 0.8037 - val_loss: 0.4268
Epoch 4/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.8277 - loss: 0.3828 - val_accuracy: 0.8137 - val_loss: 0.4081
Epoch 5/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - accuracy: 0.8533 - loss: 0.3391 - val_accuracy: 0.8110 - val_loss: 0.4091
Epoch 6/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.8710 - loss: 0.2979 - val_accuracy: 0.8115 - val_loss: 0.4161
Epoch 7/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.8979 - loss: 0.2494 - val_accuracy: 0.8205 - val_loss: 0.4471
Epoch 8/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9217 - loss: 0.1946 - val_accuracy: 0

2025-12-19 00:21:23.869980: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-12-19 00:21:24.925301: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_695', 520 bytes spill stores, 520 bytes spill loads



479/479 ━━━━━━━━━━━━━━━━━━━━ 15s 25ms/step - accuracy: 0.6691 - loss: 0.6063 - val_accuracy: 0.7328 - val_loss: 0.5321
Epoch 2/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.7524 - loss: 0.5018 - val_accuracy: 0.7808 - val_loss: 0.4586
Epoch 3/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.8017 - loss: 0.4312 - val_accuracy: 0.7785 - val_loss: 0.4651
Epoch 4/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.8302 - loss: 0.3753 - val_accuracy: 0.7834 - val_loss: 0.4609
Epoch 5/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.8517 - loss: 0.3307 - val_accuracy: 0.8456 - val_loss: 0.3498
Epoch 6/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 13s 26ms/step - accuracy: 0.8774 - loss: 0.2829 - val_accuracy: 0.8496 - val_loss: 0.3417
Epoch 7/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - accuracy: 0.9052 - loss: 0.2307 - val_accuracy: 0.8331 - val_loss: 0.3974
Epoch 8/20
479/479 ━━━━━━━━━━━━━━━━━━━━ 18s 38ms/step - accuracy: 0.9268 - loss: 0.1859 - val_accuracy

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/nigga/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/nigga/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/nigga/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 758, in start

  File "/home/nigga/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/nigga/.local/lib/python3.10/site-packages/ipykernel/utils.py", line 71, i

***
Couche  	      |  Rôle<br>
***
Conv2D            | (32 filtres)	Détecte bords & formes simples<br>
***

MaxPool	          | Réduit la taille<br>
***
Conv2D(64 filtres)|	Détecte motifs plus complexes<br>
***
MaxPool           |	Réduit encore<br>
***
Flatten	          |Convertit en vecteur<br>
***
Dense             |(64)	Apprend les combinaisons de features<br>
***
Dense (1, sigmoid)|	Prédit 0 ou 1<br>
***